# Stock Market Data Analysis

This notebook connects to BigQuery and visualizes the stock market data that we've transformed using dbt. We'll create several visualizations to analyze trends and patterns in the data.

## Setup and Configuration

First, let's install and import the necessary libraries.

In [ ]:
# Install required packages
!pip install pandas matplotlib seaborn plotly google-cloud-bigquery pandas-gbq

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.cloud import bigquery

# Set plotting style
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Make plots larger and more readable
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

## Connect to BigQuery

Now, let's set up the connection to BigQuery. Make sure you have the service account key file available.

In [ ]:
# Set your GCP project ID
PROJECT_ID = "your-gcp-project-id"  # Replace with your actual project ID

# Initialize BigQuery client
# Note: This assumes you've already authenticated with GCP
# If not, you can use: client = bigquery.Client.from_service_account_json('path/to/key.json')
client = bigquery.Client(project=PROJECT_ID)

## Query and Visualize Monthly Performance Data

Let's start by querying the monthly performance data from our mart model.

In [ ]:
# Query monthly performance data
monthly_performance_query = """
SELECT
  month,
  avg_sp500_price,
  avg_nasdaq_price,
  avg_tech_price,
  avg_commodity_price,
  avg_crypto_price
FROM
  `stock_market_data_marts.stock_performance`
ORDER BY
  month ASC
"""

monthly_performance = client.query(monthly_performance_query).to_dataframe()

# Display the first few rows
monthly_performance.head()

In [ ]:
# Convert month to datetime for better plotting
monthly_performance['month'] = pd.to_datetime(monthly_performance['month'])

# Create a time series plot of major indices
plt.figure(figsize=(14, 8))
plt.plot(monthly_performance['month'], monthly_performance['avg_sp500_price'], label='S&P 500', linewidth=2)
plt.plot(monthly_performance['month'], monthly_performance['avg_nasdaq_price'], label='NASDAQ', linewidth=2)
plt.title('S&P 500 and NASDAQ Monthly Average Prices (2019-2024)', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Average Price', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Create an interactive plot with Plotly
fig = px.line(monthly_performance, x='month', 
              y=['avg_tech_price', 'avg_commodity_price', 'avg_crypto_price'],
              labels={
                  'month': 'Date',
                  'value': 'Average Price',
                  'variable': 'Asset Class'
              },
              title='Average Prices by Asset Class (2019-2024)',
              line_shape='spline',
              render_mode='svg')

# Update layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Average Price',
    legend_title='Asset Class',
    hovermode='x unified',
    template='plotly_white'
)

# Customize line colors and names
fig.update_traces(
    selector=dict(name='avg_tech_price'),
    name='Tech Stocks',
    line=dict(color='#1f77b4', width=3)
)
fig.update_traces(
    selector=dict(name='avg_commodity_price'),
    name='Commodities',
    line=dict(color='#ff7f0e', width=3)
)
fig.update_traces(
    selector=dict(name='avg_crypto_price'),
    name='Cryptocurrencies',
    line=dict(color='#2ca02c', width=3)
)

fig.show()

## Analyze Tech Stock Performance

Let's query and analyze the performance of major tech stocks.

In [ ]:
# Query tech stock performance data
tech_stocks_query = """
SELECT
  month,
  avg_apple_price,
  avg_microsoft_price,
  avg_google_price,
  avg_amazon_price,
  avg_meta_price,
  avg_nvidia_price,
  avg_tesla_price
FROM
  `stock_market_data_marts.stock_performance`
ORDER BY
  month ASC
"""

tech_stocks = client.query(tech_stocks_query).to_dataframe()

# Convert month to datetime
tech_stocks['month'] = pd.to_datetime(tech_stocks['month'])

# Display the first few rows
tech_stocks.head()

In [ ]:
# Create a multi-line plot for tech stocks
plt.figure(figsize=(16, 10))

# Plot each tech stock
plt.plot(tech_stocks['month'], tech_stocks['avg_apple_price'], label='Apple', linewidth=2)
plt.plot(tech_stocks['month'], tech_stocks['avg_microsoft_price'], label='Microsoft', linewidth=2)
plt.plot(tech_stocks['month'], tech_stocks['avg_google_price'], label='Google', linewidth=2)
plt.plot(tech_stocks['month'], tech_stocks['avg_amazon_price'], label='Amazon', linewidth=2)
plt.plot(tech_stocks['month'], tech_stocks['avg_meta_price'], label='Meta', linewidth=2)
plt.plot(tech_stocks['month'], tech_stocks['avg_nvidia_price'], label='NVIDIA', linewidth=2)

plt.title('Tech Stock Performance (2019-2024)', fontsize=18)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Average Price ($)', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Create an interactive heatmap of tech stock correlations
# First, calculate correlations
tech_corr = tech_stocks.drop('month', axis=1).corr()

# Create a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(tech_corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1, linewidths=0.5)
plt.title('Correlation Between Tech Stocks', fontsize=16)
plt.tight_layout()
plt.show()

## Analyze Commodity and Cryptocurrency Performance

Now, let's look at commodities and cryptocurrencies.

In [ ]:
# Query commodity and crypto performance data
commodities_crypto_query = """
SELECT
  month,
  avg_gold_price,
  avg_crude_oil_price,
  avg_natural_gas_price,
  avg_bitcoin_price,
  avg_ethereum_price
FROM
  `stock_market_data_marts.stock_performance`
ORDER BY
  month ASC
"""

commodities_crypto = client.query(commodities_crypto_query).to_dataframe()

# Convert month to datetime
commodities_crypto['month'] = pd.to_datetime(commodities_crypto['month'])

# Display the first few rows
commodities_crypto.head()

In [ ]:
# Create subplots for commodities
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12), sharex=True)

# Plot commodities
ax1.plot(commodities_crypto['month'], commodities_crypto['avg_gold_price'], label='Gold', color='gold', linewidth=2)
ax1.plot(commodities_crypto['month'], commodities_crypto['avg_crude_oil_price'], label='Crude Oil', color='black', linewidth=2)
ax1.set_title('Commodity Prices (2019-2024)', fontsize=16)
ax1.set_ylabel('Price ($)', fontsize=14)
ax1.legend(fontsize=12)
ax1.grid(True)

# Plot cryptocurrencies
ax2.plot(commodities_crypto['month'], commodities_crypto['avg_bitcoin_price'], label='Bitcoin', color='orange', linewidth=2)
ax2.plot(commodities_crypto['month'], commodities_crypto['avg_ethereum_price'], label='Ethereum', color='blue', linewidth=2)
ax2.set_title('Cryptocurrency Prices (2019-2024)', fontsize=16)
ax2.set_xlabel('Date', fontsize=14)
ax2.set_ylabel('Price ($)', fontsize=14)
ax2.legend(fontsize=12)
ax2.grid(True)

plt.tight_layout()
plt.show()

## Analyze Relationships Between Different Asset Classes

Let's explore the relationships between different asset classes.

In [ ]:
# Create a correlation matrix for all asset classes
# Combine all dataframes
all_assets = pd.merge(monthly_performance, tech_stocks, on='month')
all_assets = pd.merge(all_assets, commodities_crypto, on='month')

# Select key columns for correlation analysis
correlation_columns = [
    'avg_sp500_price', 'avg_nasdaq_price', 'avg_tech_price', 'avg_commodity_price', 'avg_crypto_price',
    'avg_gold_price', 'avg_crude_oil_price', 'avg_bitcoin_price', 'avg_ethereum_price',
    'avg_apple_price', 'avg_microsoft_price', 'avg_nvidia_price'
]

# Calculate correlations
asset_corr = all_assets[correlation_columns].corr()

# Create a heatmap
plt.figure(figsize=(16, 14))
sns.heatmap(asset_corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1, linewidths=0.5, fmt='.2f')
plt.title('Correlation Between Different Asset Classes', fontsize=18)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Create Interactive Scatter Plots

Let's create some interactive scatter plots to explore relationships between different assets.

In [ ]:
# Create an interactive scatter plot with Plotly
fig = px.scatter(all_assets, x='avg_sp500_price', y='avg_gold_price', 
                 size='avg_bitcoin_price', color='avg_tech_price',
                 hover_name='month', 
                 labels={
                     'avg_sp500_price': 'S&P 500 Price',
                     'avg_gold_price': 'Gold Price',
                     'avg_bitcoin_price': 'Bitcoin Price',
                     'avg_tech_price': 'Tech Stock Price'
                 },
                 title='Relationship Between S&P 500, Gold, and Bitcoin Prices',
                 size_max=50)

fig.update_layout(
    xaxis_title='S&P 500 Price',
    yaxis_title='Gold Price',
    coloraxis_colorbar_title='Tech Stock Price',
    template='plotly_white'
)

fig.show()

## Analyze Performance During Key Market Events

Let's identify and analyze performance during key market events like the COVID-19 pandemic.

In [ ]:
# Define key market events
events = {
    'COVID-19 Crash': '2020-03-01',
    'Recovery Start': '2020-04-01',
    '2022 Market Downturn': '2022-01-01',
    '2023 Recovery': '2023-01-01'
}

# Convert event dates to datetime
events = {k: pd.to_datetime(v) for k, v in events.items()}

# Create a plot with event markers
plt.figure(figsize=(16, 10))

# Plot S&P 500 and NASDAQ
plt.plot(monthly_performance['month'], monthly_performance['avg_sp500_price'], label='S&P 500', linewidth=2)
plt.plot(monthly_performance['month'], monthly_performance['avg_nasdaq_price'], label='NASDAQ', linewidth=2)

# Add event markers
for event, date in events.items():
    plt.axvline(x=date, color='red', linestyle='--', alpha=0.7)
    plt.text(date, plt.ylim()[1]*0.95, event, rotation=90, verticalalignment='top')

plt.title('Market Performance During Key Events (2019-2024)', fontsize=18)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Average Price', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

## Create a Dashboard-like Summary

Finally, let's create a dashboard-like summary with multiple plots.

In [ ]:
# Create a dashboard with Plotly subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'S&P 500 and NASDAQ Performance',
        'Tech Stocks Performance',
        'Commodities Performance',
        'Cryptocurrencies Performance'
    ),
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Add S&P 500 and NASDAQ traces
fig.add_trace(
    go.Scatter(x=monthly_performance['month'], y=monthly_performance['avg_sp500_price'], name='S&P 500'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=monthly_performance['month'], y=monthly_performance['avg_nasdaq_price'], name='NASDAQ'),
    row=1, col=1
)

# Add tech stock traces
fig.add_trace(
    go.Scatter(x=tech_stocks['month'], y=tech_stocks['avg_apple_price'], name='Apple'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=tech_stocks['month'], y=tech_stocks['avg_microsoft_price'], name='Microsoft'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=tech_stocks['month'], y=tech_stocks['avg_nvidia_price'], name='NVIDIA'),
    row=1, col=2
)

# Add commodity traces
fig.add_trace(
    go.Scatter(x=commodities_crypto['month'], y=commodities_crypto['avg_gold_price'], name='Gold'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=commodities_crypto['month'], y=commodities_crypto['avg_crude_oil_price'], name='Crude Oil'),
    row=2, col=1
)

# Add cryptocurrency traces
fig.add_trace(
    go.Scatter(x=commodities_crypto['month'], y=commodities_crypto['avg_bitcoin_price'], name='Bitcoin'),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=commodities_crypto['month'], y=commodities_crypto['avg_ethereum_price'], name='Ethereum'),
    row=2, col=2
)

# Update layout
fig.update_layout(
    title_text='Stock Market Dashboard (2019-2024)',
    height=800,
    width=1200,
    showlegend=True,
    template='plotly_white'
)

# Update x-axis labels
fig.update_xaxes(title_text='Date', row=2, col=1)
fig.update_xaxes(title_text='Date', row=2, col=2)

# Update y-axis labels
fig.update_yaxes(title_text='Price ($)', row=1, col=1)
fig.update_yaxes(title_text='Price ($)', row=1, col=2)
fig.update_yaxes(title_text='Price ($)', row=2, col=1)
fig.update_yaxes(title_text='Price ($)', row=2, col=2)

fig.show()

## Conclusion

In this notebook, we've created various visualizations to analyze the stock market data from 2019-2024. We've looked at:

1. Performance of major indices (S&P 500, NASDAQ)
2. Tech stock performance and correlations
3. Commodity and cryptocurrency trends
4. Relationships between different asset classes
5. Performance during key market events

These visualizations provide insights into market trends and can help inform investment decisions.